# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print 'Training set:\t{0} candidates'.format(len(train))
print 'Dev set:\t{0} candidates'.format(len(dev))
print 'Test set:\t{0} candidates'.format(len(test))    

Training set:	8272 candidates
Dev set:	888 candidates
Test set:	4620 candidates


In [47]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, split=0)

# Computing features

For `SparseLogReg`

In [5]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

%time F_train = featurizer.apply(split=0)
F_train

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 6min 27s, sys: 2.28 s, total: 6min 29s
Wall time: 6min 30s


<8272x120876 sparse matrix of type '<type 'numpy.float64'>'
	with 444009 stored elements in Compressed Sparse Row format>

In [6]:
%%time
F_dev  = featurizer.apply_existing(split=1)
F_test = featurizer.apply_existing(split=2)

Clearing existing...
Running UDF...
[========================================] 100%

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 4min 1s, sys: 1.88 s, total: 4min 3s
Wall time: 4min 2s


# Training `SparseLogReg`

Instead of LSTM, to start.  First, reloading features:

In [48]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

In [49]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, F_train, train_marginals, [rate_param, l1_param, l2_param], n=20)

Initialized RandomSearch search of size 20. Search space size = 125.


In [50]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

<888x1 sparse matrix of type '<type 'numpy.float64'>'
	with 888 stored elements in Compressed Sparse Row format>

In [51]:
import numpy as np
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=0.5, print_freq=25)

[1] Testing lr = 1.00e-02, l1_penalty = 1.00e-03, l2_penalty = 1.00e-04
[SparseLR] lr=0.01 l1=0.001 l2=0.0001
[SparseLR] Building model
[SparseLR] Training model
[SparseLR] #examples=6960  #epochs=50  batch size=100
[SparseLR] Epoch 0 (7.90s)	Avg. loss=0.582203	NNZ=120800
[SparseLR] Epoch 25 (17.22s)	Avg. loss=0.416255	NNZ=120827
[SparseLR] Epoch 49 (26.21s)	Avg. loss=0.405611	NNZ=120788
[SparseLR] Training done (26.21s)
[SparseLR] Model saved. To load, use name
		SparseLR_0
[2] Testing lr = 1.00e-04, l1_penalty = 1.00e-06, l2_penalty = 1.00e-03
[SparseLR] lr=0.0001 l1=1e-06 l2=0.001
[SparseLR] Building model
[SparseLR] Training model
[SparseLR] #examples=6960  #epochs=50  batch size=100
[SparseLR] Epoch 0 (1.20s)	Avg. loss=0.685407	NNZ=120876
[SparseLR] Epoch 25 (10.47s)	Avg. loss=0.583310	NNZ=120876
[SparseLR] Epoch 49 (19.38s)	Avg. loss=0.559029	NNZ=120876
[SparseLR] Training done (19.38s)
[3] Testing lr = 1.00e-03, l1_penalty = 1.00e-05, l2_penalty = 1.00e-05
[SparseLR] lr=0.001 l1

,lr,l1_penalty,l2_penalty,Prec.,Rec.,F1
7,0.010000,0.000010,0.010000,0.444954,0.655405,0.530055
4,0.010000,0.000100,0.000010,0.454988,0.631757,0.528996
19,0.010000,0.000010,0.000100,0.452555,0.628378,0.526167
0,0.010000,0.001000,0.000100,0.444712,0.625000,0.519663
2,0.001000,0.000010,0.000010,0.444712,0.625000,0.519663
17,0.010000,0.000010,0.000001,0.446078,0.614865,0.517045
3,0.001000,0.000001,0.001000,0.437647,0.628378,0.515950
11,0.001000,0.000001,0.001000,0.436620,0.628378,0.515235
13,0.001000,0.000010,0.000100,0.434368,0.614865,0.509091
15,0.000010,0.000001,0.000100,0.449198,0.567568,0.501493


### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

In [33]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')

AnnotatorLabels created: 0


In [12]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

<4620x1 sparse matrix of type '<type 'numpy.float64'>'
	with 4620 stored elements in Compressed Sparse Row format>

In [52]:
_, _, _, _ = disc_model.score(session, F_test, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.702
Neg. class accuracy: 0.581
Precision            0.447
Recall               0.702
F1                   0.546
----------------------------------------
TP: 1056 | FP: 1306 | TN: 1809 | FN: 449



# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

In [ ]:
from snorkel.contrib.learning import reLSTM

lstm = reLSTM()
lstm.train(
    train, train_marginals, lr=0.005, dim=200, n_epochs=30,
    dropout_rate=0.5, rebalance=0.25, print_freq=5
)